In [5]:
!pip install qiskit>=1.0 qiskit-aer>=0.14

In [17]:
pip install qiskit-aer

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

# Parameters
L = 1.0; T = 0.1; nu = 0.05; n_qubits = 3; N = 2**n_qubits

# Step 1: Cole-Hopf transform (initial condition)
x = np.linspace(0, L, N, endpoint=False)
phi0 = np.exp(-np.where(x <= L/2, x/(2*nu), (L-x)/(2*nu)))

# Step 2: State preparation with proper normalization
norm = np.linalg.norm(phi0)
psi0 = phi0 / norm  # Normalize the state vector

# Quantum circuit setup
qc = QuantumCircuit(n_qubits)
qc.initialize(psi0, range(n_qubits))

# Step 3: Quantum evolution (QFT + phase gates)
qc.append(QFT(n_qubits), range(n_qubits))

# Apply exp(-i * ν * t * k^2) for each k
for k in range(N):
    theta = -nu * T * (k**2)
    # Convert k to binary representation and apply phase to corresponding state
    for qubit in range(n_qubits):
        if (k >> qubit) & 1:
            qc.p(theta, qubit)

qc.append(QFT(n_qubits, inverse=True), range(n_qubits))

# Simulation using Statevector
statevector = Statevector.from_instruction(qc)
probs = statevector.probabilities()

# Step 4: Postprocessing
phi_final = np.sqrt(probs) * norm  # Reconstruct ϕ
u = -2*nu * np.gradient(phi_final, x) / phi_final  # Compute u(x,T)

print("Final velocity field u(x,T):", u)

Final velocity field u(x,T): [ 0.43483733  0.6348292   0.40213607 -0.03206207 -0.07706119  0.00296724
  0.07396298  0.4670324 ]


C:\Users\Suptotthita\AppData\Local\Temp\ipykernel_16156\2969718529.py:23: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits), range(n_qubits))
C:\Users\Suptotthita\AppData\Local\Temp\ipykernel_16156\2969718529.py:33: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_qubits, inverse=True), range(n_qubits))
